# BLAST based check for the BP model - biological validation

Comparing the predictions obtained and the blast annotation. 

In [46]:
import pandas as pd

In [47]:
# Loading the BLAST annotation, train set

blast = pd.read_csv('data/test/blast_test_results.tsv', sep = '\t')

train_set = pd.read_csv('data/train/train_set.tsv', sep = "\t")

train_bp = train_set[train_set['aspect'] == 'biological_process']



In [48]:
# lookup table
# train protein ID to list of known BP GO terms
bp_map = (
    train_bp
    .groupby("Protein_ID")["GO_term"]
    .apply(list)
    .to_dict()
)

blast_bp_preds = []
TOP_N = 5 # number of top blast hits to consider

for pid, hits in blast.groupby("query"):                         # loop over test proteins and their blast hits
    hits = hits.sort_values("bits", ascending=False).head(TOP_N) # sort by blast bit score
    max_bits = hits["bits"].max()                                # normalize according to best hit for the query

    for _, row in hits.iterrows():                               # for each hit take train protein match
        for go in bp_map.get(row["target"], []):                 # retrieve all its BP GO terms
            blast_bp_preds.append(                               # asign BP term to test protein with a normalized confidence score
                (pid, go, row["bits"] / max_bits)
            )

In [49]:
# convert BP preds into a structured df
blast_bp_df = pd.DataFrame(
    blast_bp_preds,
    columns=["Protein_ID", "GO_term", "score"]
)

# keep highest confidence score if BP term is infered multiple times for the same protein
blast_bp_df = blast_bp_df.groupby(
    ["Protein_ID", "GO_term"], as_index=False
)["score"].max()

In [50]:
# loading ML based predictions
bp_predictions = pd.read_csv(
    "metadata/BP/bp_predictions.csv"
)

# combine ml and blast preds
final_bp = pd.concat([bp_predictions, blast_bp_df])

# if same term appears from both, keep the highest confidence term
final_bp = final_bp.groupby(
    ["Protein_ID", "GO_term"], as_index=False
)["score"].max()

In [51]:
# remove zeroes
final_bp = final_bp[final_bp["score"] > 0]

# term sort, desc by conf, per protein, keep top 500 (max terms is way lower)
final_bp = (
    final_bp
    .sort_values(["Protein_ID", "score"], ascending=[True, False])
    .groupby("Protein_ID")
    .head(500)  # MF-only cap
)

In [52]:
# global summary statistics

print("Total BP predictions:", len(final_bp))
final_bp.groupby("Protein_ID").size().describe()

Total BP predictions: 178267


count    1000.000000
mean      178.267000
std        44.459536
min        55.000000
25%       153.000000
50%       168.000000
75%       194.000000
max       441.000000
dtype: float64

In [53]:
# inspecting a single protein part 1
final_bp[final_bp["Protein_ID"] == final_bp["Protein_ID"].iloc[0]].head(10)

,Protein_ID,GO_term,score
0,A0A0B4JCV4,GO:0000003,1.0
2,A0A0B4JCV4,GO:0000226,1.0
3,A0A0B4JCV4,GO:0000278,1.0
4,A0A0B4JCV4,GO:0000280,1.0
9,A0A0B4JCV4,GO:0003006,1.0
11,A0A0B4JCV4,GO:0006810,1.0
12,A0A0B4JCV4,GO:0006996,1.0
13,A0A0B4JCV4,GO:0006997,1.0
14,A0A0B4JCV4,GO:0007010,1.0
15,A0A0B4JCV4,GO:0007017,1.0


In [54]:
# convert GO identifiers into biological terms
GO_OBO_PATH = "data/train/go-basic.obo"

go_names = {}

with open(GO_OBO_PATH, "r") as f:
    current_id = None
    for line in f:
        line = line.strip()
        if line.startswith("id: GO:"):
            current_id = line.split("id: ")[1]
        elif line.startswith("name:") and current_id:
            go_names[current_id] = line.split("name: ")[1]
            current_id = None

print("Loaded GO terms:", len(go_names))

Loaded GO terms: 47637


In [55]:
#check
go_names.get("GO:0005634")  # should be "nucleus"

'nucleus'

In [56]:
# inspecting single protein predictions part 2
pid = final_bp["Protein_ID"].iloc[0]

inspect = (
    final_bp[final_bp["Protein_ID"] == pid]
    .sort_values("score", ascending=False)
    .head(10)
)

inspect["GO_name"] = inspect["GO_term"].map(go_names)
inspect
# they should be closely related terms

,Protein_ID,GO_term,score,GO_name
0,A0A0B4JCV4,GO:0000003,1.0,reproduction
2,A0A0B4JCV4,GO:0000226,1.0,microtubule cytoskeleton organization
3,A0A0B4JCV4,GO:0000278,1.0,mitotic cell cycle
4,A0A0B4JCV4,GO:0000280,1.0,nuclear division
9,A0A0B4JCV4,GO:0003006,1.0,developmental process involved in reproduction
11,A0A0B4JCV4,GO:0006810,1.0,transport
12,A0A0B4JCV4,GO:0006996,1.0,organelle organization
13,A0A0B4JCV4,GO:0006997,1.0,nucleus organization
14,A0A0B4JCV4,GO:0007010,1.0,cytoskeleton organization
15,A0A0B4JCV4,GO:0007017,1.0,microtubule-based process


In [57]:
# global GO name distribution, root terms should have all test proteins (1000)
# shpuld decrease steadily at each few levels
final_bp["GO_name"] = final_bp["GO_term"].map(go_names)

final_bp["GO_name"].value_counts()

GO_name
biological_process                                 1000
cellular process                                   1000
response to stimulus                                995
biological regulation                               988
regulation of biological process                    980
                                                   ... 
reciprocal meiotic recombination                      5
reciprocal homologous recombination                   5
glycosaminoglycan metabolic process                   4
compound eye photoreceptor cell differentiation       4
evasion of host immune response                       2
Name: count, Length: 1487, dtype: int64

In [58]:
# inspecting single protein term distribution =  they should all be 1, no duplicates
inspect["GO_name"].apply(lambda x: x.lower()).value_counts()

GO_name
reproduction                                      1
microtubule cytoskeleton organization             1
mitotic cell cycle                                1
nuclear division                                  1
developmental process involved in reproduction    1
transport                                         1
organelle organization                            1
nucleus organization                              1
cytoskeleton organization                         1
microtubule-based process                         1
Name: count, dtype: int64

In [59]:
# MF terms predicted by blast but not the ML model
# for n number of proteins, blast assigned the x term, but the Ml model didn't
# they should be pretty high level terms

blast_only = blast_bp_df.merge(
    bp_predictions,
    on=["Protein_ID", "GO_term"],
    how="left",
    indicator=True
)

blast_only = blast_only[blast_only["_merge"] == "left_only"]

blast_only["GO_name"] = blast_only["GO_term"].map(go_names)

blast_only["GO_name"].value_counts().head(10)

GO_name
negative regulation of signal transduction          84
negative regulation of response to stimulus         83
negative regulation of cell communication           79
regulation of intracellular signal transduction     78
negative regulation of signaling                    78
regulation of molecular function                    77
positive regulation of protein metabolic process    76
regulation of apoptotic process                     73
cellular response to organic substance              73
regulation of immune system process                 73
Name: count, dtype: int64

In [60]:
# global comparison
ml_terms = set(bp_predictions["GO_term"])
blast_terms = set(blast_bp_df["GO_term"])

shared_terms = ml_terms & blast_terms
ml_only_terms = ml_terms - blast_terms
blast_only_terms = blast_terms - ml_terms

print("ML terms total:", len(ml_terms))
print("BLAST terms total:", len(blast_terms))
print("Shared terms:", len(shared_terms))
print("ML-only terms:", len(ml_only_terms))
print("BLAST-only terms:", len(blast_only_terms))

ML terms total: 1451
BLAST terms total: 1487
Shared terms: 1451
ML-only terms: 0
BLAST-only terms: 36


In [61]:
# checking the ML only term

ml_only_terms = ml_terms - blast_terms
len(ml_only_terms), ml_only_terms

(0, set())

In [62]:
# mapping to names
# expecting specific subcelular structures that BLAST does not transfer and vice-versa
ml_only_df = (
    pd.DataFrame({"GO_term": list(ml_only_terms)})
)
ml_only_df["GO_name"] = ml_only_df["GO_term"].map(go_names)
ml_only_df

,GO_term,GO_name


In [64]:
# checking the blast only terms

blast_only_df = (
    pd.DataFrame({"GO_term": list(blast_only_terms)})
)
blast_only_df["GO_name"] = blast_only_df["GO_term"].map(go_names)
blast_only_df.sort_values("GO_name")

# found some protein complexes which ML will miss unless all component proteins are present in the training

,GO_term,GO_name
33,GO:0002250,adaptive immune response
23,GO:0007189,adenylate cyclase-activating G protein-coupled...
0,GO:0008356,asymmetric cell division
20,GO:0048754,branching morphogenesis of an epithelial tube
3,GO:0019722,calcium-mediated signaling
16,GO:1901655,cellular response to ketone
8,GO:0048546,digestive tract morphogenesis
25,GO:0050673,epithelial cell proliferation
7,GO:0022612,gland morphogenesis
34,GO:0021782,glial cell development


In [65]:
# terms that only Blast assigned, Blast specific terms and in how many proteins it appears
# extremely rare, specific and specialized terms expected, ML can't pick them up
# many low numbers, most terms should have 1 protein (toggle head to check)

blast_only_counts = (
    blast_bp_df[blast_bp_df["GO_term"].isin(blast_only_terms)]
    .groupby("GO_term")["Protein_ID"]
    .nunique()
    .reset_index(name="protein_count")
    .merge(blast_only_df, on="GO_term")
    .sort_values("protein_count", ascending=False)
)

blast_only_counts

,GO_term,protein_count,GO_name
24,GO:0051098,28,regulation of binding
1,GO:0002573,26,myeloid leukocyte differentiation
8,GO:0010632,26,regulation of epithelial cell migration
28,GO:0070374,23,positive regulation of ERK1 and ERK2 cascade
2,GO:0002702,22,positive regulation of production of molecular...
20,GO:0045637,22,regulation of myeloid cell differentiation
27,GO:0061138,21,morphogenesis of a branching epithelium
11,GO:0019722,19,calcium-mediated signaling
18,GO:0032946,19,positive regulation of mononuclear cell prolif...
22,GO:0048754,19,branching morphogenesis of an epithelial tube
